# Ансамбли

### AI Masters, "Машинное обучение 2"

В этом ноутбуке вам предлагается реализовать алгоритмы бэггинга.

## 1. Сэмплирование случайных объектов и признаков

Во многих ансамблевых алгоритмах используется прием, заключающийся в обучении на случайной подвыборке объектов или на случайном подмножестве признаков.

Так что для начала реализуем класс, который будет упрощать семплирование различных подмассивов данных

В классе `ObjectSampler` надо реализовать метод `sample`, который возвращает случайную подвыборку объектов обучения и ответы для них

В классе `FeaturesSampler` надо реализовать метод `sample`, который возвращает случайную подвыборку индексов признаков, по которым будет производится обучение, а также метод `sample_by_indicies`, который для матрицы объект-признак возвращает подматрицу, в которой содержатся только выбранные признаки

## 2. Бэггинг

Суть бэггинга заключается в обучении нескольких 'слабых' базовых моделей и объединении их в одну модель, обладающую бОльшей обобщающей способностью. Каждая базовая модель обучается на случайно выбранном подмножестве объектов и на случайно выбранном подмножестве признаков для этих объектов.

Ниже вам предлагается реализовать несколько методов класса `Bagger`:
* `fit` - обучение базовых моделей
* `predict_proba` - вычисление вероятностей ответов
* `predict` - вычисление ответов

## 3. Библиотеки

In [6]:
from sklearn.datasets import make_blobs
import numpy as np

In [7]:
X, y = make_blobs(n_samples=200000, n_features=20, cluster_std=2, centers=5, random_state=0)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [4]:
clf = RandomForestClassifier(n_estimators=100, max_depth=8, random_state=0)

In [5]:
%%time
clf.fit(X, y)

CPU times: user 40.4 s, sys: 412 ms, total: 40.9 s
Wall time: 41.5 s


RandomForestClassifier(max_depth=8, random_state=0)

In [3]:
import lightgbm as lgb
import numpy as np

In [8]:
params = {
    'n_estimators': 100,
    'num_leaves': 2 ** 8,
    'min_data_in_leaf': 10,
    'objective': 'multiclass',
    'num_classes': len(np.unique(y)),
    'max_depth': 8,
    "boosting": "rf",
    "seed": 0,
    "bagging_frequency": 0.65,
    "subsample": .632,
    "subsample_freq": 1,
    "verbose": -1,
}

In [9]:
data = lgb.Dataset(data=X, label=y)

In [10]:
%%time
clf = lgb.train(params, train_set=data, num_boost_round=100)

: 

In [11]:
dir(clf)

['_Booster__boost',
 '_Booster__get_eval_info',
 '_Booster__higher_better_inner_eval',
 '_Booster__init_predictor',
 '_Booster__inner_eval',
 '_Booster__inner_predict',
 '_Booster__inner_predict_buffer',
 '_Booster__is_predicted_cur_iter',
 '_Booster__name_inner_eval',
 '_Booster__need_reload_eval_info',
 '_Booster__num_class',
 '_Booster__num_dataset',
 '_Booster__num_inner_eval',
 '_Booster__set_objective_to_none',
 '__class__',
 '__copy__',
 '__deepcopy__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_free_buffer',
 '_get_loaded_param',
 '_handle',
 '_network',
 '_train_data_name',
 'add_valid',
 'best_iteration',
 'best_score',


In [11]:
import xgboost as xgb

In [8]:
data = xgb.DMatrix(X, label=y)


In [10]:
X.shape, y.shape

((200000, 20), (200000,))

In [6]:
params = {
    "booster": "gbtree",
    "subsample": .632,
    "max_depth": 8,
    "num_parallel_tree": 100,
    "objective": "multi:softmax",
    "num_class": len(np.unique(y)),
    "eta":1,
    "random_state": 0,
    "n_jobs": -1,
}

In [7]:
%%time
bst = xgb.train(params, data, num_boost_round=1)